In [ ]:
import pandas as pd
IMAGE_PATH = "./data_source/images/"
TEST_PATH = "./data_source/test.csv"
test_data = pd.read_csv(TEST_PATH)
SUB_PATH = "./data_source/sample_submission.csv"
sub = pd.read_csv(SUB_PATH)
from Model import get_model

In [ ]:
nets = ['MobileNet','EfficientNetB61','EfficientNetB62','Inception','ResNet','DenseNet201']
bin_nets = ['BinaryModelB6']
CHECKPOINTS_BASE = 'checkpoints'
weights = {
    'MobileNet': '%s/MobileNet/best_model_val_loss.h5'%CHECKPOINTS_BASE,
    'DenseNet': '%s/DenseNet/best_model_val_loss_0.18127.h5'%CHECKPOINTS_BASE,
    'DenseNet201': '%s/DenseNet201/best_model_val_loss.h5'%CHECKPOINTS_BASE,
    'Inception': '%s/Inception/best_model_val_loss.h5'%CHECKPOINTS_BASE,
    'ResNet': '%s/ResNet/best_model_val_loss.h5'%CHECKPOINTS_BASE,
    'EfficientNetB61': '%s/effnet/best_model.hdf5'%CHECKPOINTS_BASE,
    'EfficientNetB62': '%s/effnet/best_model_1.8477e-06.hdf5'%CHECKPOINTS_BASE,
    'BinaryModelB6': '%s/b6/best_model_f1.hdf5'%CHECKPOINTS_BASE}

models = []
for net in nets:
    model = get_model(net)
    model.load_weights(weights.get(net))
    models.append(model)

bin_models = []
bin_models.append(get_model(bin_nets[0]))
bin_models[0].load_weights(weights.get(bin_nets[0]))

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from matplotlib import image
from numpy import asarray
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


X_train = []
Y_train = []

size = (299,299)
class_map = {'healthy':0,'multiple_diseases':1,'rust':2,'scab':3}

def load_image(image_id,size):  
    file_path = image_id
    with Image.open(file_path) as image:
        image = image.resize(size)
    return np.asarray(image)

import glob
for img_path in glob.glob("data/Test/*/*.jpg"):
    Y_train.append(class_map.get(img_path.split('\\')[1]))
    X_train.append(load_image(img_path,size) / 255.)
X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [ ]:
def average_preds(p,indices):
    preds = models[0].predict(p)
    preds.fill(0)
    for i in range(len(models)):
        if i in indices:
            preds = preds + models[i].predict(p)
    preds = preds / len(indices)
    preds_classes = []
    for pred in preds:
        preds_classes.append(np.argmax(pred))
    return preds_classes

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from matplotlib import image
from numpy import asarray
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def load_image(image_id,size):  
    file_path = image_id + ".jpg"
    with Image.open(IMAGE_PATH + file_path) as image:
        image = image.resize(size)
    return np.asarray(image)

IMAGE_PATH = "./data_source/images/"
TEST_PATH = "./data_source/test.csv"
test_data = pd.read_csv(TEST_PATH)


size = (image_size[1], image_size[0])

# Test data of the challenge (no y values)
print("Loading Test Images")
X_test = []
progress = 0
for image_id in test_data["image_id"]:
    X_test.append(load_image(image_id,size) / 255.)
    print('Loading Test Images: %f%%'%((progress/len(test_data["image_id"])*100)))
    progress+=1
    clear_output(wait=True)
X_test = np.array(X_test)

In [ ]:
X_train = []
Y_train = []

class_map = {'healthy':0,'multiple_diseases':1,'rust':2,'scab':3}

def load_image(image_id,size):  
    file_path = image_id
    with Image.open(file_path) as image:
        image = image.resize(size)
    return np.asarray(image)

import glob
for img_path in glob.glob("data/Test/*/*.jpg"):
    Y_train.append(class_map.get(img_path.split('\\')[1]))
    X_train.append(load_image(img_path,size) / 255.)
X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [ ]:
def average_preds(p):
    preds = models[0].predict(p,verbose=1)
    for i in range(1,len(models)):
        preds = preds + models[i].predict(p,verbose=1)
    preds = preds / len(models)
    return preds

def ensemble(preds,mutli_prop,multi_importance = 0.8):
    for i in range(len(preds)):
        pred = preds[i]
        multi = mutli_prop[i]
        #if multi > 0.8:
        for j in range(len(pred)):
            if j==1:
                preds[i][j] = (preds[i][j]*(1-multi_importance)+(multi*multi_importance))
            else:
                preds[i][j] = (preds[i][j]*(1-multi_importance)+(1-multi)*multi_importance)
    return preds
preds = ensemble(average_preds(X_train),bin_models[0].predict(X_train),multi_importance=0)
preds_single = []
for pred in preds:
    preds_single.append(np.argmax(pred))

confusion_matrix(Y_train,preds_single)

In [ ]:
def load_image(image_id,size):  
    file_path = image_id + ".jpg"
    with Image.open(IMAGE_PATH + file_path) as image:
        image = image.resize(size)
    return np.asarray(image)


print("Loading Test Images")
X_test = []
progress = 0
for image_id in test_data["image_id"]:
    X_test.append(load_image(image_id,size) / 255.)
    print('Loading Test Images: %f%%'%((progress/len(test_data["image_id"])*100)))
    progress+=1
    clear_output(wait=True)
X_test = np.array(X_test)

In [ ]:
def average_preds(p,indices):
    preds = models[0].predict(p)
    preds.fill(0)
    for i in range(len(models)):
        if i in indices:
            preds = preds + models[i].predict(p)
    preds = preds / len(indices)
    return preds

def ensemble(preds,mutli_prop):
    multi_importance = 0.8
    for i in range(len(preds)):
        pred = preds[i]
        multi = mutli_prop[i]
        if multi > 0.8:
            for j in range(len(pred)):
                if j==1:
                    preds[i][j] = (preds[i][j]*(1-multi_importance)+(multi*multi_importance))
                else:
                    preds[i][j] = (preds[i][j]*(1-multi_importance)+(1-multi)*multi_importance)
    return preds

def LabelSmoothing(encodings , alpha=0.01):
    K = encodings.shape[1]
    y_ls = (1 - alpha) * encodings + alpha / K
    return y_ls

In [ ]:
multi_importance = 0.575
probs_efnns = average_preds(X_test,[2,3])
probs_efnns = LabelSmoothing(probs_efnns)
sub.loc[:, 'healthy':] = probs_efnns
sub.to_csv('./submission_efnns.csv', index=False)
sub.head()